<a href="https://colab.research.google.com/github/Duppal147/Hackathon2025/blob/main/hackathon_pre_game_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LINK: https://www.kaggle.com/datasets/nathanlauga/nba-games

To build a machine learning model for predicting NBA game outcomes using historical Kaggle data, follow these steps:

Data Collection and Preparation
Obtain the NBA dataset from Kaggle, which includes comprehensive information on teams, players, and games28.

Clean and preprocess the data:

Remove any irrelevant or redundant features

Handle missing values

Encode categorical variables

Normalize numerical features

Create relevant features:

Calculate team performance metrics (e.g., average points scored, rebounds, assists)

Compute player statistics

Generate features based on recent performance (e.g., last 10 games)3

Feature Selection
Identify key performance indicators that influence game outcomes:

Field goal percentage

Defensive rebounds

Turnovers

Assists

Three-point shooting percentage4

Use feature importance techniques like correlation analysis or SHAP (SHapley Additive exPlanations) to select the most relevant features4.

Model Selection and Training
Split the data into training and testing sets (e.g., 80:20 ratio)3.

Choose and implement machine learning algorithms:

Logistic Regression

Random Forest Classifier

XGBoost Classifier

Support Vector Classifier

Gaussian Naïve Bayes14

Train the models using the training data.

Perform hyperparameter tuning using techniques like grid search or Bayesian optimization4.

Model Evaluation
Evaluate model performance using metrics such as:

Accuracy

Precision

Recall

F1 Score

AUC (Area Under the Curve)4

Use cross-validation techniques (e.g., 10-fold cross-validation) to ensure robust performance assessment4.

Model Refinement
Analyze feature importance to understand which factors contribute most to the predictions4.

Consider ensemble methods or stacking to combine multiple models for improved performance.

Implement techniques like rolling averages or time-based features to capture recent team performance3.

Deployment and Prediction
Select the best-performing model based on evaluation metrics.

Implement the model in a production environment.

Use the model to predict outcomes of upcoming NBA games by inputting the latest team and player statistics.

Continuously monitor and update the model with new data to maintain its accuracy over time.

By following these steps, you can create a machine learning model to predict NBA game outcomes using historical Kaggle data. Remember that the accuracy of such models typically ranges from 65% to 70%14, so while they can provide valuable insights, they are not perfect predictors of game results.




In [1]:
# Importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
import os

In [2]:
games = pd.read_csv('https://raw.githubusercontent.com/Duppal147/Hackathon2025/refs/heads/main/games.csv')
rankings=pd.read_csv("https://raw.githubusercontent.com/Duppal147/Hackathon2025/refs/heads/main/ranking.csv")
teams = pd.read_csv('https://raw.githubusercontent.com/Duppal147/Hackathon2025/refs/heads/main/teams.csv')

In [3]:
games.columns


Index(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID',
       'VISITOR_TEAM_ID', 'SEASON', 'TEAM_ID_home', 'PTS_home', 'FG_PCT_home',
       'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'TEAM_ID_away',
       'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
       'REB_away', 'HOME_TEAM_WINS'],
      dtype='object')

In [4]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26651 entries, 0 to 26650
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GAME_DATE_EST     26651 non-null  object 
 1   GAME_ID           26651 non-null  int64  
 2   GAME_STATUS_TEXT  26651 non-null  object 
 3   HOME_TEAM_ID      26651 non-null  int64  
 4   VISITOR_TEAM_ID   26651 non-null  int64  
 5   SEASON            26651 non-null  int64  
 6   TEAM_ID_home      26651 non-null  int64  
 7   PTS_home          26552 non-null  float64
 8   FG_PCT_home       26552 non-null  float64
 9   FT_PCT_home       26552 non-null  float64
 10  FG3_PCT_home      26552 non-null  float64
 11  AST_home          26552 non-null  float64
 12  REB_home          26552 non-null  float64
 13  TEAM_ID_away      26651 non-null  int64  
 14  PTS_away          26552 non-null  float64
 15  FG_PCT_away       26552 non-null  float64
 16  FT_PCT_away       26552 non-null  float6

In [5]:
games['GAME_DATE_EST'] = pd.to_datetime(games['GAME_DATE_EST'])

y=Hometeamwins is the y what you are trying to predict for the model, so it would be the y

Field G

In [ ]:
# Check for missing values in each dataframe
print("Missing values in games.csv:\n", games.isnull().sum())


Missing values in games.csv:
 GAME_DATE_EST        0
GAME_ID              0
GAME_STATUS_TEXT     0
HOME_TEAM_ID         0
VISITOR_TEAM_ID      0
SEASON               0
TEAM_ID_home         0
PTS_home            99
FG_PCT_home         99
FT_PCT_home         99
FG3_PCT_home        99
AST_home            99
REB_home            99
TEAM_ID_away         0
PTS_away            99
FG_PCT_away         99
FT_PCT_away         99
FG3_PCT_away        99
AST_away            99
REB_away            99
HOME_TEAM_WINS       0
dtype: int64


In [6]:
columns_to_check = [
    'PTS_home', 'FG_PCT_home', 'FG3_PCT_home', 'FT_PCT_home', 'AST_home', 'REB_home',
    'FG_PCT_away', 'FG3_PCT_away', 'FT_PCT_away', 'AST_away', 'REB_away', 'PTS_away'
]

games = games.dropna(subset=columns_to_check)


In [7]:
games.isnull().sum()

,0
GAME_DATE_EST,0
GAME_ID,0
GAME_STATUS_TEXT,0
HOME_TEAM_ID,0
VISITOR_TEAM_ID,0
SEASON,0
TEAM_ID_home,0
PTS_home,0
FG_PCT_home,0
FT_PCT_home,0


In [ ]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26552 entries, 0 to 26650
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GAME_DATE_EST     26552 non-null  object 
 1   GAME_ID           26552 non-null  int64  
 2   GAME_STATUS_TEXT  26552 non-null  object 
 3   HOME_TEAM_ID      26552 non-null  int64  
 4   VISITOR_TEAM_ID   26552 non-null  int64  
 5   SEASON            26552 non-null  int64  
 6   TEAM_ID_home      26552 non-null  int64  
 7   PTS_home          26552 non-null  float64
 8   FG_PCT_home       26552 non-null  float64
 9   FT_PCT_home       26552 non-null  float64
 10  FG3_PCT_home      26552 non-null  float64
 11  AST_home          26552 non-null  float64
 12  REB_home          26552 non-null  float64
 13  TEAM_ID_away      26552 non-null  int64  
 14  PTS_away          26552 non-null  float64
 15  FG_PCT_away       26552 non-null  float64
 16  FT_PCT_away       26552 non-null  float64
 17

In [ ]:

print("Initial Data Info:")
print(rankings.info())
print("\nFirst 5 rows:")
#rankings = rankings.drop(['CONFERENCE','RETURNTOPLAY'], axis=1)
#already removed Conference and Return to Play
print(rankings)

#Need to change Home_record and Road_record to integers
print(rankings)
print(games)

# Need to figure out a way to not drop standings date and team, but not use it while making our model

Initial Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210342 entries, 0 to 210341
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TEAM_ID        210342 non-null  int64  
 1   LEAGUE_ID      210342 non-null  int64  
 2   SEASON_ID      210342 non-null  int64  
 3   STANDINGSDATE  210342 non-null  object 
 4   CONFERENCE     210342 non-null  object 
 5   TEAM           210342 non-null  object 
 6   G              210342 non-null  int64  
 7   W              210342 non-null  int64  
 8   L              210342 non-null  int64  
 9   W_PCT          210342 non-null  float64
 10  HOME_RECORD    210342 non-null  object 
 11  ROAD_RECORD    210342 non-null  object 
 12  RETURNTOPLAY   3990 non-null    float64
dtypes: float64(2), int64(6), object(5)
memory usage: 20.9+ MB
None

First 5 rows:
           TEAM_ID  LEAGUE_ID  SEASON_ID STANDINGSDATE CONFERENCE  \
0       1610612743          0      22022   

In [ ]:
games.columns

Index(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID',
       'VISITOR_TEAM_ID', 'SEASON', 'TEAM_ID_home', 'PTS_home', 'FG_PCT_home',
       'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'TEAM_ID_away',
       'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
       'REB_away', 'HOME_TEAM_WINS'],
      dtype='object')

In [8]:
shortened_games = games[['GAME_DATE_EST', 'GAME_ID', 'SEASON', 'TEAM_ID_home', 'TEAM_ID_away', 'PTS_home', 'FG_PCT_home','FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away','REB_away', 'HOME_TEAM_WINS']]
games_21 = shortened_games[shortened_games['SEASON'] == 2021]
games_21


,GAME_DATE_EST,GAME_ID,SEASON,TEAM_ID_home,TEAM_ID_away,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
542,2022-06-16,42100406,2021,1610612738,1610612744,90.0,0.425,0.917,0.393,27.0,41.0,103.0,0.413,1.000,0.413,27.0,44.0,0
543,2022-06-13,42100405,2021,1610612744,1610612738,104.0,0.466,0.867,0.225,23.0,39.0,94.0,0.413,0.677,0.344,18.0,47.0,1
544,2022-06-10,42100404,2021,1610612738,1610612744,97.0,0.400,0.737,0.395,22.0,42.0,107.0,0.440,0.800,0.349,20.0,55.0,0
545,2022-06-08,42100403,2021,1610612738,1610612744,116.0,0.483,0.708,0.371,28.0,47.0,100.0,0.462,0.867,0.375,22.0,31.0,1
546,2022-06-05,42100402,2021,1610612744,1610612738,107.0,0.453,0.700,0.405,25.0,42.0,88.0,0.375,0.765,0.405,24.0,43.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1926,2021-10-04,12100007,2021,1610612759,1610612762,111.0,0.465,0.760,0.353,22.0,60.0,85.0,0.317,0.600,0.233,14.0,47.0,1
1927,2021-10-04,12100008,2021,1610612757,1610612744,107.0,0.446,0.773,0.421,21.0,53.0,121.0,0.420,0.765,0.348,32.0,47.0,0
1928,2021-10-04,12100009,2021,1610612758,1610612756,117.0,0.473,0.684,0.340,22.0,51.0,106.0,0.467,0.846,0.314,24.0,38.0,1
1929,2021-10-04,12100010,2021,1610612746,1610612743,103.0,0.432,0.760,0.333,26.0,52.0,102.0,0.430,0.621,0.270,20.0,45.0,1


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Create a simple feature: average ranking difference
rankings['win_ratio'] = rankings['W'] / (rankings['W'] + rankings['L'])
# Merge ranking data with games data
games = pd.merge(games, rankings[['TEAM_ID', 'win_ratio']], left_on='HOME_TEAM_ID', right_on='TEAM_ID', suffixes=('', '_home'))
games = pd.merge(games, rankings[['TEAM_ID', 'win_ratio']], left_on='visitor_team_id', right_on='TEAM_ID', suffixes=('_home', '_away'))

# Calculate ranking difference
games['ranking_diff'] = games['win_ratio_home'] - games['win_ratio_away']

# Define target variable (home team win)
games['home_win'] = (games['home_team_score'] > games['visitor_team_score']).astype(int)

# Point differential
games['score_diff'] = games['PTS_home'] - games['PTS_away']  # Difference in points scored

# Shooting percentage differentials
games['fg_pct_diff'] = games['FG_PCT_home'] - games['FG_PCT_away']  # Field Goal % difference
games['ft_pct_diff'] = games['FT_PCT_home'] - games['FT_PCT_away']  # Free Throw % difference
games['fg3_pct_diff'] = games['FG3_PCT_home'] - games['FG3_PCT_away']  # 3-Point % difference

# Rolling averages for the last 5 games they played
for stat in ['PTS', 'FG_PCT', 'FT_PCT', 'FG3_PCT', 'AST', 'REB']:
    for team_type in ['home', 'away']:
        games[f'{stat}_{team_type}_last_5_avg'] = games.groupby(f'TEAM_ID_{team_type}')[f'{stat}_{team_type}'].rolling(window=5, min_periods=1).mean().reset_index(0, drop=True)

# Select features and target
X = games[['ranking_diff']]  # Use only ranking difference as a feature
y = games['home_win']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Model Training
model = LogisticRegression()
model.fit(X_train, y_train)

# Model Evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


In [ ]:
print("Columns in games DataFrame:")
print(games.columns)

print("\nColumns in rankings DataFrame:")
print(rankings.columns)

In [9]:
rankings_21 = rankings[rankings['SEASON_ID'] == 22021]
rankings_21

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
1260,1610612756,0,22021,2022-09-29,West,Phoenix,82,64,18,0.780,32-9,32-9,NaN
1261,1610612763,0,22021,2022-09-29,West,Memphis,82,56,26,0.683,30-11,26-15,NaN
1262,1610612744,0,22021,2022-09-29,West,Golden State,82,53,29,0.646,31-10,22-19,NaN
1263,1610612742,0,22021,2022-09-29,West,Dallas,82,52,30,0.634,29-12,23-18,NaN
1264,1610612762,0,22021,2022-09-29,West,Utah,82,49,33,0.598,29-12,20-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15070,1610612752,0,22021,2021-10-19,East,New York,0,0,0,0.000,0-0,0-0,NaN
15071,1610612753,0,22021,2021-10-19,East,Orlando,0,0,0,0.000,0-0,0-0,NaN
15072,1610612755,0,22021,2021-10-19,East,Philadelphia,0,0,0,0.000,0-0,0-0,NaN
15073,1610612761,0,22021,2021-10-19,East,Toronto,0,0,0,0.000,0-0,0-0,NaN


In [10]:
home_stats = games_21.copy()
home_stats = home_stats.rename(columns={
    'TEAM_ID_home': 'TEAM_ID',
    'PTS_home': 'PTS',
    'FG_PCT_home': 'FG_PCT',
    'FT_PCT_home': 'FT_PCT',
    'FG3_PCT_home': 'FG3_PCT',
    'AST_home': 'AST',
    'REB_home': 'REB',
    'TEAM_ID_away': 'OPP_ID',
    'PTS_away': 'OPP_PTS',
    'FG_PCT_away': 'OPP_FG_PCT',
    'FT_PCT_away': 'OPP_FT_PCT',
    'FG3_PCT_away': 'OPP_FG3_PCT',
    'AST_away': 'OPP_AST',
    'REB_away': 'OPP_REB'
})
home_stats = home_stats.reset_index(drop=True)

away_stats = games_21.copy()
away_stats = away_stats.rename(columns={
    'TEAM_ID_away': 'TEAM_ID',
    'PTS_away': 'PTS',
    'FG_PCT_away': 'FG_PCT',
    'FT_PCT_away': 'FT_PCT',
    'FG3_PCT_away': 'FG3_PCT',
    'AST_away': 'AST',
    'REB_away': 'REB',
    'TEAM_ID_home': 'OPP_ID',
    'PTS_home': 'OPP_PTS',
    'FG_PCT_home': 'OPP_FG_PCT',
    'FT_PCT_home': 'OPP_FT_PCT',
    'FG3_PCT_home': 'OPP_FG3_PCT',
    'AST_home': 'OPP_AST',
    'REB_home': 'OPP_REB',
})
away_stats = away_stats.reset_index(drop=True)

combined_stats = pd.concat([home_stats, away_stats], ignore_index=True)

team_avgs = combined_stats.groupby('TEAM_ID')[['PTS', 'FG_PCT', 'FT_PCT', 'FG3_PCT', 'AST', 'REB', 'OPP_PTS', 'OPP_FG_PCT', 'OPP_FT_PCT', 'OPP_FG3_PCT', 'OPP_AST',	'OPP_REB']].mean().reset_index()
team_avgs

,TEAM_ID,PTS,FG_PCT,FT_PCT,FG3_PCT,AST,REB,OPP_PTS,OPP_FG_PCT,OPP_FT_PCT,OPP_FG3_PCT,OPP_AST,OPP_REB
0,1610612737,112.709677,0.467753,0.809817,0.369462,24.268817,43.870968,111.494624,0.468860,0.795355,0.361720,25.301075,43.731183
1,1610612738,110.090909,0.461618,0.813773,0.356418,24.681818,45.354545,104.054545,0.435955,0.783591,0.337909,21.463636,43.545455
2,1610612739,107.337079,0.467876,0.761449,0.352258,25.089888,44.000000,105.831461,0.453315,0.766483,0.351236,24.247191,43.168539
3,1610612740,108.914894,0.456787,0.783766,0.336830,24.553191,45.468085,110.276596,0.473638,0.792330,0.359894,25.000000,41.702128
4,1610612741,110.989011,0.475187,0.808363,0.362165,24.010989,42.868132,111.197802,0.471780,0.788297,0.361615,24.857143,43.769231
5,1610612742,108.134615,0.460308,0.774817,0.355010,22.903846,42.442308,104.471154,0.458827,0.767212,0.340337,23.423077,43.807692
6,1610612743,112.336957,0.480739,0.794457,0.351315,27.478261,44.021739,110.804348,0.471913,0.750859,0.348326,25.532609,42.336957
7,1610612744,111.495413,0.470734,0.771587,0.366193,27.211009,45.275229,105.788991,0.438872,0.760661,0.346954,23.110092,42.752294
8,1610612745,109.511628,0.455453,0.710128,0.343419,23.441860,42.418605,118.104651,0.482570,0.781791,0.349826,25.604651,45.744186
9,1610612746,108.068182,0.456057,0.787727,0.369807,23.977273,44.113636,108.715909,0.451500,0.761443,0.341080,24.147727,47.420455
